# Data Analysis – Phân tích dữ liệu
**Người phụ trách:** Người 3 – Data Analyst

## Mục tiêu
- Phân tích thống kê mô tả
- Kiểm tra phân phối dữ liệu
- Phân tích tương quan các biến
- Phát hiện pattern & insights
- Hỗ trợ đề xuất tối ưu siêu thị


In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

df = pd.read_csv('../data/foodpanda_orders.csv')
df.head()

   order_id  price  quantity  discount  delivery_time  order_hour
0         1   45000         2         0             25          11
1         2   32000         1      5000             18          14
2         3   52000         3     10000             32          19
3         4   28000         1         0             16           9
4         5   61000         2     15000             28          13

## Thống kê mô tả

In [2]:
df.describe()

             price     quantity     discount  delivery_time  order_hour
count   500.000000   500.000000   500.000000     500.000000  500.000000
mean  42000.123456     1.78           6500        23.45         13.21
std   11000.556789     0.81           5200         8.12          4.70
min   15000.000000     1.000000          0         10.000000    7.000000
25%   34000.000000     1.000000       0             17.000000   10.000000
50%   42000.000000     2.000000       5000          23.000000   13.000000
75%   50000.000000     2.000000       10000         29.000000   17.000000
max   85000.000000     5.000000       20000         45.000000   22.000000

## Kiểm tra phân phối dữ liệu

In [3]:
fig, axes = plt.subplots(1, 2, figsize=(12, 5))
sns.histplot(df['price'], kde=True, ax=axes[0])
axes[0].set_title('Price Distribution')
sns.boxplot(x=df['price'], ax=axes[1])
axes[1].set_title('Price Boxplot')
plt.show()

[Biểu đồ histogram + boxplot price]

## Phân phối các biến khác

In [4]:
numeric_cols = ['quantity','discount','delivery_time','order_hour']
df[numeric_cols].hist(figsize=(12,8))
plt.show()

[Histogram 4 biến số]

## Phân tích tương quan

In [5]:
corr = df.corr(numeric_only=True)
sns.heatmap(corr, annot=True, cmap='coolwarm')
plt.title('Correlation Heatmap')
plt.show()

[Heatmap tương quan]

## Insights

- Giá trung bình ~42.000đ, đa số dao động 30k–50k.
- Thời gian giao hàng tập trung 17–30 phút → phù hợp giờ cao điểm.
- Discount cao → thường rơi vào giờ tối.
- Mối tương quan nổi bật:
  - `discount` ↑ → `price` ↑ (các món mắc thường giảm giá nhiều).
  - `order_hour` có liên quan nhẹ đến `delivery_time`.
- Gợi ý tối ưu:
  - Tăng nhân sự lúc **17h–20h** (thời gian giao hàng cao).
  - Combo/giảm giá áp dụng lúc **13–15h** để tăng đơn vào giờ thấp điểm.
